In [ ]:
from map import Map, Marker
import db

In [ ]:
boundaries = db.query(
    """
select st_asgeojson(geom) from boundaries
where id = 'chicago' 
"""
)

In [ ]:
wards = db.query(
    """
select ward, st_asgeojson(geom) from stg_chicago__wards
"""
)

In [ ]:
from ipyleaflet import GeoJSON
import json

m = Map((41.83, -87.5))

for row in boundaries.fetchall():
    geo_json = GeoJSON(data=json.loads(row[0]), style={"color": "red", "weight": 5})
    m.add(geo_json)

for row in wards.fetchall():
    geo_json = GeoJSON(
        data=json.loads(row[1]),
        style={"weight": 1},
        hover_style={"fillOpacity": 0.5},
    )
    m.add(geo_json)

m